# Regularized Benders Decomposition - Multi-Cut
[Murwan Siddig](mailto:msiddig@clemson.edu)


### Two-stage stochastic LP (with fixed recourse):

\begin{aligned}
\displaystyle \min_{x\in\mathbb{R}^{n_1}_+, y\in\mathbb{R}^{n_2}_+} & c^\intercal x+ \mathbb{E}_{\xi}[q_\xi^\intercal y_\xi] \\
\mbox{s.t.} & Ax &\geq b \\
            & T_\xi x +W y_\xi &\geq h_\xi \\
\end{aligned}


where
* $A \in \mathbb{R}^{m_1\times n_1}$
* $T_\xi \in \mathbb{R}^{m_2\times n_1}$
* $W \in \mathbb{R}^{m_2\times n_2}$
* $c \in \mathbb{R}^{n_1}$
* $q_\xi \in \mathbb{R}^{n_2}$
* $b \in \mathbb{R}^{m_1}$
* $h_\xi \in \mathbb{R}^{m_2}$

### Feasiblity subproblem:
\begin{equation}
F(\hat x,\xi^s):= \left\{
\begin{array}{llll}
\displaystyle \min_{y_\xi, v^+_\xi,v^-_\xi\in\mathbb{R}^{n_2}_+} & {\bf 1}^\intercal (v^+_\xi+v^-_\xi) \\
\mbox{s.t.} & W y_\xi + v^+_\xi-v^-_\xi\geq h_\xi-T_\xi \hat x \\
\end{array}
\right.
\equiv
\left\{
\begin{array}{llll}
\displaystyle \max_{\lambda_\xi\in\mathbb{R}^{m_2}_+} & (h_\xi-T_\xi \hat x)^\intercal\lambda_\xi \\
\mbox{s.t.} & W^\intercal \lambda_\xi &\leq 0\\
            & +\lambda_\xi &\leq {\bf 1}\\
            & -\lambda_\xi &\leq {\bf 1}\\
\end{array}
\right. 
\end{equation}

### Optimality subproblem:
\begin{equation}
Q(\hat x,\xi^s):= \left\{
\begin{array}{llll}
\displaystyle \min_{y_\xi\in\mathbb{R}^{n_2}_+} & q_\xi^\intercal y_\xi \\
\mbox{s.t.} & W y_\xi \geq h_\xi-T_\xi \hat x \\
\end{array}
\right.
\equiv
\left\{
\begin{array}{llll}
\displaystyle \max_{\pi_\xi\in\mathbb{R}^{m_2}_+} &(h_\xi-T_\xi \hat x)^\intercal\pi_\xi \\
\mbox{s.t.} & W^\intercal \pi_\xi \leq q_\xi\\
\end{array}
\right.
\end{equation}


### Master problem (multi-cut):
\begin{aligned}
\displaystyle \min_{x\in\mathbb{R}^{n_1}_+,\,\; \theta \in \mathbb{R}^{|\Xi|}} & \displaystyle c^\intercal x+ \sum_{\forall \xi \in \Xi}\theta_{\xi}\times p_\xi \\
\mbox{s.t.} & Ax \geq b \\
            & 0\geq \displaystyle (h_\xi-T_\xi x)^\intercal\lambda^j_\xi\times p_\xi & \forall \; \xi \in \Xi,\; j=1, \dots, J_{\xi,f}\\
            & \theta_\xi \geq \displaystyle (h_\xi-T_\xi x)^\intercal\pi^j_\xi\times p_\xi & \forall \; \xi \in \Xi,\;  j=1, \dots, J_{\xi,o}\\
\end{aligned}


where 
* $J_{\xi,f}$ is the number of scenario $\xi$ feasiblity cuts added.
* $J_{\xi,o}$ is the number of scenario $\xi$ optimality cuts added.


### Regularized Master Problem (multi-cut):

\begin{aligned}
\displaystyle \min_{x\in\mathbb{R}^{n_1}_+, \theta\in\mathbb{R}} & c^\intercal x+ \sum_{\forall \xi \in \Xi}\theta_{\xi}\times p_\xi + \frac{1}{2}||x - \check x||^2 \\
\mbox{s.t.} & c^\intercal x+ \sum_{\forall \xi \in \Xi}\theta_{\xi}\times p_\xi \leq f_{\text{lev}}\\
            & Ax \geq b \\
            & 0\geq \displaystyle (h_\xi-T_\xi \hat x)^\intercal\lambda^j_\xi & \forall \; j=1, \dots, J_f, \forall \xi \in \Xi\\
            & \theta^\xi \geq \displaystyle (h_\xi-T_\xi x)^\intercal\pi^j_\xi & \forall \; j=1, \dots, J_o, \forall \xi \in \Xi\\
\end{aligned}


where 
* $\check x$ is the incumbent solution. 
* $f_{\text{lev}} := f_{\text{low}} + \gamma(f_{\text{up}}-f_{\text{low}})$ is the level set parameter and $\gamma \in (0,1)$.


In [1]:
import numpy as np
import pandas as pd
from operator import add
import gurobipy as gp;
from gurobipy import GRB;
import numpy as np;

In [2]:
A = [[1, 1, 0, 0]];
T = [[1.0, 1.0, 0.0, 0.0],
     [1.0, 1.0, 0.0, 0.0],
     [0.5, 0.0, 1.0, 0.0],
     [0.5, 0.0, 0.0, 1.0]];
W = [[1, 0, 0, 0],
     [0, 1, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 0, 1]];
c = [[1], [2], [2], [1]];
q = [[1], [2], [1], [2]];
b = [[4]];
h = [[7], [4], [5], [8]];

In [3]:
n1 = len(c); # number of 1st-stage variables
n2 = len(q); # number of 2nd-stage variables
m1 = len(b); # number of 1st-stage constraints
m2 = len(h); # number of 2nd-stage constraints

In [4]:
Ξ = [[1,1/3],
     [4,1],
     [5/2,2/3],
     [1,1]];
N = len(Ξ);

P = [1/N for k in range(N)];

ϵ = 1e-5; #tolarence parameter 

In [5]:
#define functions to modify T according to scenario k
def Tₓ(T,Ξ,n1,m2,k):
    ξ = Ξ[k];
    temp = np.ones((m2,n1));
    temp[0][0] = ξ[0];
    temp[1][0] = ξ[1];
    temp[2][0] = ξ[0];
    temp[3][0] = ξ[1];      
    temp[3][3] = ξ[0];
    return T*temp

#define functions to modify q according to scenario k
def qₓ(q,Ξ,n2,k):
    return q

#define functions to modify h according to scenario k
def hₓ(h,Ξ,m2,k):
    return h

# Solve a mean value problem (MVP) to find an initial lower bound for θ:

In [6]:
ξ̄ = [sum(Ξ[k][n]*P[k] for k in range(N)) for n in range(len(Ξ[0]))];
q̄ = q;
h̄ = h;
T̄ = Tₓ(T,[ξ̄],n1,m2,0)

MVP = gp.Model("MVP");
MVP.modelSense = GRB.MINIMIZE;

x̄ = {};
for j in range(n1):
   x̄[j] = MVP.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=c[j][0]);

ȳ = {};
for j in range(n2):
    ȳ[j] = MVP.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=q̄[j][0]);
    
for i in range(m1):
    MVP.addConstr(sum(A[i][j]*x̄[j] for j in range(n1))>= b[i][0]);
    
for i in range(m2):
    MVP.addConstr(sum(T̄[i][j]*x̄[j] for j in range(n1))+sum(W[i][j]*ȳ[j] for j in range(n2))>= h[i][0]);
    
MVP.update();
MVP.setParam("OutputFlag", 0);
print("===============================================================");
MVP.optimize();
if MVP.status == GRB.OPTIMAL:
    Z̲ = MVP.objVal
    print('MVP objective Z̄ = %g' % Z̲);
    
    x̄val = {};
    for j in range(n1):
        x̄val[j]=x̄[j].x
    
    ȳval = {};
    for j in range(n2):
        ȳval[j]=ȳ[j].x;
else:
    print('No solution');
    print('status = ', MVP.status);

Using license file /Users/murwansiddig/gurobi.lic
Academic license - for non-commercial use only
MVP objective Z̄ = 8.15686


In [7]:
#=================================================================================================================
# Optimality subproblem 
sub_opt = gp.Model("subprob_optimality");
sub_opt.modelSense = GRB.MINIMIZE; 

yᵒ = {};
for j in range(n2):
    yᵒ[j] = sub_opt.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=q[j][0]);

Π = {};
for i in range(m2):
    Π[i] = sub_opt.addConstr(sum(W[i][j]*yᵒ[j] for j in range(n2))>= 0);
    
sub_opt.setParam("OutputFlag", 0);
#=================================================================================================================

#=================================================================================================================
# Feasibility subproblem 
sub_feas = gp.Model("subprob_feasibility");
sub_feas.modelSense = GRB.MINIMIZE; 

yᶠ = {};
v̅ = {};
v̲ = {};

for j in range(n2):
    yᶠ[j] = sub_feas.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=0);
    
for i in range(m2):
    v̅[i] = sub_feas.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=1);
    v̲[i] = sub_feas.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=1);
    
Λ = {};
for i in range(m2):
    Λ[i] = sub_feas.addConstr(sum(W[i][j]*yᶠ[j] for j in range(n2))+v̅[i]-v̲[i] >= 0);
sub_feas.setParam("OutputFlag", 0);
#=================================================================================================================

In [10]:
#define the level parameter
def f_level(f̲,f̅,γ):
    return f̲+γ*(f̅-f̲)

In [ ]:
#=================================================================================================================
# Master problem  
Master_multi = gp.Model("Master_single");
Master_multi.modelSense = GRB.MINIMIZE;
xᵐ = {};
for j in range(n1):
   xᵐ[j] = Master_multi.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=c[j][0]);

θᵐ = {};
for k in range(N):
    θᵐ[k] = Master_multi.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, obj=P[k]);
    
#Lower bound for θ
Master_multi.addConstr(sum(c[j][0]*xᵐ[j] for j in range(n1))+sum(θᵐ[k]*P[k] for k in range(N))>= Z̲);

x̂ˢ = np.zeros(n1); #initialize the first stage decision values;
θ̂ˢ = np.zeros(N);  #initialize the second stage expected cost;
Master_multi.setParam("OutputFlag", 0);
#=================================================================================================================

LBᵐ = 0;
UBᵐ = 1e10;

Ocuts = np.zeros(N);
Fcuts = 0;

iter = 0;
while (UBᵐ-LBᵐ)*1.0/UBᵐ > ϵ:
    iter+=1;

    # First solve the master problem, and get x value and θ
    x̂ᵐ = np.zeros(n1); #initialize the first stage decision values;
    θ̂ᵐ = np.zeros(N); #initialize the second stage expected cost;
    Master_multi.optimize();
    for j in range(n1):
        x̂ᵐ[j]=xᵐ[j].x #value of x
        
    for k in range(N):
        θ̂ᵐ[k] = θᵐ[k].x; #value of θ
    
    LBᵐ = Master_multi.objVal; #update the lower bound.
    
    π = np.zeros((N,m2)); #initialize the optimality dual multipliers (assuming x̂ is feasible ∀ ξ)
    λ = np.zeros((N,m2)); #initialize the feasibility dual multipliers (in case needed);
    Q = np.zeros(N); #initialize a vector for the optimality subproblem optimal objective value ∀ ξ,
    w = np.zeros(N); #initialize a vector for the feasibility subproblem optimal objective value ∀ ξ,
    FLAG = 0; #(0 if x̂ is feasible ∀ ξ, and 1 otherwise)
    for k in range(N):
        Tᵏ = Tₓ(T,Ξ,n1,m2,k);
        qᵏ = qₓ(q,Ξ,n2,k);
        hᵏ = hₓ(h,Ξ,m2,k)
        for i in range(m2):
            Π[i].setAttr(GRB.Attr.RHS, hᵏ[i][0]-sum(Tᵏ[i][j]*x̂ᵐ[j] for j in range(n1)));
            
        sub_opt.optimize();
        if sub_opt.status == GRB.OPTIMAL:
            Q[k] = sub_opt.objVal;
            for i in range(m2):
                π[k][i] = Π[i].pi;
        elif sub_opt.status == GRB.INFEASIBLE or sub_opt.status == GRB.INF_OR_UNBD:
            FLAG = 1;
            break;
        else:
            print('One of the scenario subproblems is neither optimal or infeasible');
            print('status = ', sub_opt.status);
            break
            exit(0)
            
    if FLAG == 1:
        for k in range(N):
            Tᵏ = Tₓ(T,Ξ,n1,m2,k);
            qᵏ = qₓ(q,Ξ,n2,k);
            hᵏ = hₓ(h,Ξ,m2,k)
            for i in range(m2):
                Λ[i].setAttr(GRB.Attr.RHS, hᵏ[i][0]-sum(Tᵏ[i][j]*x̂ᵐ[j] for j in range(n1)));
                
            sub_feas.optimize(); 
            w[k] = sub_feas.objVal; #should always be optimal --  no need to check the status
            for i in range(m2):
                λ[k][i] = Λ[i].pi;
            if w[k] > ϵ:
                #Add the feasibility cut  
                Master_multi.addConstr(0>= sum(λ[k][i]*(hᵏ[i][0]-sum(Tᵏ[i][j]*xᵐ[j] for j in range(n1))) for i in range(m2)));
                Fcuts +=1;
                break
                
    else:
        θ̄ᵐ = {};
        for k in range(N):
            θ̄ᵐ[k]=Q[k];
        if sum(c[j][0]*x̂ᵐ[j] for j in range(n1))+sum(θ̄ᵐ[k]*P[k] for k in range(N)) < UBᵐ:
            UBᵐ = sum(c[j][0]*x̂ᵐ[j] for j in range(n1))+sum(θ̄ᵐ[k]*P[k] for k in range(N));
        if (UBᵐ-LBᵐ)/max(1e-10,abs(LBᵐ)) < ϵ:
            break;
        else:
            for k in range(N):
                Tᵏ = Tₓ(T,Ξ,n1,m2,k);
                qᵏ = qₓ(q,Ξ,n2,k);
                hᵏ = hₓ(h,Ξ,m2,k)
                if (θ̄ᵐ[k]-θ̂ᵐ[k])/max(1e-10,abs(θ̂ᵐ[k])) >= ϵ: 
                    #Add the optimality cut cut
                    Master_multi.addConstr(θᵐ[k]>=
                                            sum(π[k][i]*(hᵏ[i][0]-sum(Tᵏ[i][j]*xᵐ[j] for j in range(n1))) for i in range(m2)) 
                                           ); 
            Ocuts +=1;
    print("iter = ", iter);
    print("FLAG = ", FLAG)
    print("LB = ", LBᵐ);
    print("UB = ", UBᵐ);
    print("===============================================================");
    
print("#########################################################################################")
print("#########################################################################################")
print("iter = ", iter);
print("# of feasibility cuts added= ", Fcuts);
print("# of optimality cuts added = ", Ocuts);
print("LBˢ = ", LBᵐ);
print("UBˢ = ", UBᵐ);
print("x̂ˢ = ", x̂ᵐ);